# COLAB para facilitar el acceso y descarga de firebase


## Ordenado de las columnas, renonombre, ajuste el timestamp a la hora local y remplazo de posibles faltantes de informacion 

In [ ]:
import re
import pandas as pd
from datetime import datetime
from firebase_admin import firestore
from firebase_admin import credentials
import firebase_admin
!pip install firebase-admin


keyFire = {
    # pegar key de la app firebase o subir json
}

cred = credentials.Certificate(keyFire)
firebase_admin.initialize_app(credential=cred)
db = firestore.client()

estudiantes = list(db.collection(u'ingresantes').stream())

estudiantes_dict = list(map(lambda x: x.to_dict(), estudiantes))
df = pd.DataFrame(estudiantes_dict)

reordenar = ['creadoEn', 'apellido', 'nombre', 'documento', 'tipoDocumento', 'cursandoSecu',
             'email', 'edad', 'fechaNacimiento',   'sexoBio', 'direccion', 'provincia', 'departamento', 'localidad',  'queresIngresar',  'fechaFinSecu',
             'seminarioRecurs', 'celular', 'empresaCel', 'acalracionEmpresa', 'familiaEstudios',
             'tEmail',  'tNombre', 'tApellido', 'tTelefono',
             'diponibilidad',  'viviendaPropia', 'conectividad', 'trabajo', 'referenciaEntropia', 'aclaracionEntropia',
             'cue', 'anexoCue', 'iEmail', 'iSector', 'iNombre', 'iCodigoDepa', 'iDomicilio',
             'iCodigoLocal', 'iPais', 'iProvincia', 'iDepartamento', 'iLocalidad', 'iAmbito',  'Itelefono', 'iCodigoTel', 'iCodigoPostal']


df = df[reordenar]

renombrar = {
    'diponibilidad': "Disponibilidad horaria",  'viviendaPropia': "Vivienda Propia", 'conectividad': "Conectividad",
    'trabajo': "Trabajo", 'referenciaEntropia': "Referencia a Entropia", 'aclaracionEntropia': "Aclaracion aReferencia a Entropia",
    'anexoCue': "anexo cue", 'cursandoSecu': "Cursando Secundaria",
    'direccion': "direccion estudiante", 'provincia': "provincia de residencia",
    'tTelefono': "telefono tutor", 'iProvincia': "provincia institucion ",
    'email': "Email Estudiante", 'iLocalidad': "localidad institucion",
    'tEmail': "Email Tutor", 'queresIngresar': "Deseo de ingreso Universidad",
    'iPais': "Institucion Pais",'acalracionEmpresa': "aclaraciones de empresa",
    'tNombre': "Nombre Tutor", 'fechaFinSecu': "fecha fin de Secundaria",
    'seminarioRecurs': "Hizo Entropia", 'celular': "Celular Estudiante", 'localidad': "Localidad residencia",
    'fechaNacimiento': "Fecha de nacimiento", 'iSector': "sector", 'familiaEstudios': "Estudios de la Familia",
    'apellido': "apellido Estudiante", 'documento': "Documento Estudiante",
    'empresaCel': "Empresa celular", 'iNombre': "Nombre de la Institucion", 'edad': "edad",
    'sexoBio': "Sexo Biologico", 'nombre': "Nombre Estudiante", 'creadoEn': "Creado el",
    'tApellido': "Apellido Tutor", 'cue': "Cue", 'iEmail': "Email Institucin", 'iCodigoDepa': "Codigo departamento Institucion",
    'iDomicilio': "Domicilio Institucion", 'iCodigoLocal': "Codigo Local Institucion", 'iDepartamento': "Departamento Institucion", 'iAmbito': "Ambito Institucion",
    'tipoDocumento': "Tipo de Documento", 'Itelefono': "Telefono Institucion", 'iCodigoTel': "Codigo telefonico Institucion", 'iCodigoPostal': "Codigo Postal Institucion"}
df.rename(columns=renombrar, inplace=True)
#df["Documento Estudiante"]=df["Documento Estudiante"].replace("[^0-9]", '', regex=True)

correccionFechas = pd.to_datetime(df['Creado el'], utc=True).dt.tz_convert(
    'Etc/GMT-3').dt.strftime('%Y-%m-%d %H:%M:%S')

df.drop(["Creado el"], axis=1, inplace=True)

df.insert(0, "Creado el", correccionFechas, True)

values = {"Institucion Pais": "Argentina"}
df.fillna(value=values, inplace=True)

#######################################################################
nombreDB = 'ingresantes'+str(datetime.today().strftime('%Y-%m-%d'))+'.csv'

with open(nombreDB, "w") as archivo:
    archivo.write(df.to_csv())


## Descargar directo

In [ ]:
from google.colab import files

files.download('tablaFirebase.csv')

## Conectar y trasferir CSV a un Drive

In [ ]:
import shutil, os
from google.colab import drive

drive.mount('/content/gdrive')
ruta= "/content/drive/My Drive/"+nombreDB # 'gdrive/My Drive/' #ruta= 'gdrive/My Drive/'+nombreDB
shutil.copy("/content/"+nombreDB,ruta)